# The advection-diffusion transport problem

This notebook contains functions to set up domain, assign boundary conditions, solve Darcy, then solve advection-diffusion transport.

Its aim is to be as modular as possible. I.e. any part can be taken out and inserted into another notebook.

## Equations

We are mainly considering the transport equation, but will also mention Darcy's law, since its solution is required for the transport problem.

#### Darcy:
$$- \nabla \cdot (K\nabla p) = f$$

where the pressure $p$ depends on $K$, the permeability, and $f$, a possible source term.

#### Transport:
$$ \rho C \left( \frac{\partial T}{\partial t} + u\cdot \nabla T \right) - \nabla\cdot(K_T\nabla T) = g$$
where $T$ may be interpreted as temperature. $\rho$ is density, $C$ is specific heat capacity, $K_T$ is thermal conductivity and $u = -K\nabla p$.

## Method
1. Create a domain
2. Assign parameters and boundary conditions for flow and transport
3. Solve Darcy
4. Compute darcy flux
5. Solve transport

In [ ]:
import porepy as pp
import numpy as np
import scipy.sparse as sps

from porepy.utils.derived_discretizations import implicit_euler as IE_discretizations

In [ ]:
def create_domain_2d(domain, mesh_args, fracs_coords, fracs):
    """ Create a fractured 2d domain.
    
    Parameters:
    domain (dict): Dictionary specifying domain boundaries.
        Assumes containing 'xmin', 'xmax', 'ymin', 'ymax'.
    mesh_args (dict): Dictorionary specifying meshing arguments
        Must contain keys:
            'mesh_size_frac': Mesh size in fractures. 
            'mesh_size_min': Minimum mesh size.
        Optional keys:
            'mesh_size_bound': Mesh size at boundaries.
    fracs_coords (np.ndarray 2 x n): Coordinates of fractures.
    fracs (np.ndarray 2 x num_fracs): Endpoints of fractures.
        Defines a mapping to fracs_coords.
            
    """
    network_2d = pp.FractureNetwork2d(fracs_coords, fracs, domain)
    gb = network_2d.mesh(mesh_args)
    return gb

In [ ]:
def dirichlet_bc(g, flowdir, domain):
    """ Assign dirichlet inflow and outflow. No-flow otherwise.
    
    Parameters
    g (pp.Grid): Grid
    flowdir (char): Flow direction. One of 'E', 'W', 'N', 'S', 
                                    ('U', 'D' for 3D).
    domain (dict): Domain. Assumes keys 'xmin', 'xmax', 'ymin', 'ymax',
                                    ('zmin', 'zmax' for 3D).
                                    
    """
    
    b_faces = g.tags['domain_boundary_faces'].nonzero()[0]
    b_face_centers = g.face_centers[:, b_faces]
    
    # Compute dirichlet flow directions
    tol = 1e-4
    if flowdir in ['W', 'E']:
        axis = 0
        mi, ma = map(domain.get, ['xmin', 'xmax'])
    elif flowdir in ['N', 'S']:
        axis = 1
        mi, ma = map(domain.get, ['ymin', 'ymax'])
    elif flowdir in ['U', 'D']:
        axis = 2
        mi, ma = map(domain.get, ['zmin', 'zmax'])

    # flowdir in ['E', 'N', 'U']
    b_inflow = b_face_centers[axis, :] < mi + tol
    b_outflow = b_face_centers[axis, :] > ma - tol

    if flowdir in ['S', 'W', 'D']:
        b_inflow, b_outflow = b_outflow, b_inflow
        
    
    # Compute geometrical indices for boundary conditions.
    labels = np.array(['neu'] * b_faces.size)
    labels[np.logical_or(b_inflow, b_outflow)] = "dir"
    bc = pp.BoundaryCondition(g, b_faces, labels)

    # Set inflow to 4. (Outflow is 0).
    bc_val = np.zeros(g.num_faces)
    bc_val[b_faces[b_inflow]] = 1
    
    return bc, bc_val


def assign_data(gb, domain, keyword, data):
    """ Assign data to a problem.
    
    The parameter keyword will either be 'flow' or 'transport' for
    default initizalization to work.
    
    Parameters
    gb (pp.GridBucket): Grid bucket
    domain (dict): Specifies the grid boundaries.
        Assumes it contains the keys: xmin, xmax, ymin, ymax, 
                        (and zmin, zmax for 3D problems).
    data (dict): Dictionary of data related to the problem.
        Recognised keywords: 
            frac_perm (float, default: 1e3): permeability in fractures
            matrix_perm (float, default: 1): permeability in matrix
            flowdir (str, default: 'E'): Direction of flow.
                Valid values: 'N', 'E', 'S', 'W', 'U', 'D'.
            porosity (float, default: 0.2): Porosity.
            dt (float): Time step
            t_max (float): End time for simulation
            advection_weight (float, 0<=w<=1): Advection weight for 
                implicit time stepping
            aperture (float): size of fractures.
            
    """
    
    # Retrieve parameters in data:
    frac_perm = data.get('frac_perm',1e3)
    matrix_perm = data.get('matrix_perm', 1)
    flowdir = data.get('flowdir', 'E')
    poro = data.get('porosity', 0.2)
    dt = data.get('time_step', 1 / 60)
    t_max = data.get("t_max", 1 / 3)
    aper = data.get('aperture', 1e-4)
    
    for g, d in gb:
        
        unity = np.ones(g.num_cells)
        # Porosity
        if g.dim == gb.dim_max():
            porosity = poro * unity
            aperture = 1
        else:
            porosity = (1 - poro) * unity
            aperture = np.power(aper, gb.dim_max() - g.dim)
        
        # Assign time_step and t_max for transport problems
        if keyword == 'transport':
            specified_parameters = {
                'time_step': dt,
                't_max': t_max,
                'mass_weight': porosity * aperture,
            }
        else:
            specified_parameters = {}
        
        # Assign permeability for matrix and fractures
        if g.dim == gb.dim_max():
            kxx = matrix_perm * np.ones(g.num_cells)
        else:
            kxx = frac_perm * np.ones(g.num_cells)
        perm = pp.SecondOrderTensor(kxx)
        specified_parameters['second_order_tensor'] = perm
        
        # Boundary conditions
        b_faces = g.tags['domain_boundary_faces'].nonzero()[0]
        bc_val = np.zeros(g.num_faces)
        
        unity = np.ones(g.num_cells)
        empty = np.empty(0)
        if b_faces.size != 0:
            bc, bc_val = dirichlet_bc(g, flowdir, domain)       
        else:
            bc = pp.BoundaryCondition(g)
        specified_parameters['bc'] = bc
        specified_parameters['bc_values'] = bc_val
        
        # Store the assigned data
        pp.initialize_default_data(g, d, keyword, specified_parameters)
        
        # Store the dimension in the dictionary for visualization purposes
        d[pp.STATE] = {"dimension": g.dim * np.ones(g.num_cells)}
    
    for e, d in gb.edges():        
        
        params = {'normal_diffusivity': 2e1}
        mg = d["mortar_grid"]
        # This method also initializes d[pp.DISCRETIZATION_MATRICES] :)
        pp.initialize_data(
            g=mg,
            data=d,
            keyword=keyword,
            specified_parameters=params
        )
    
    return gb

# Set up keywords and variables for the equations

PorePy handles equations in a really comprehensive manner. For simple problems, it may seem a bit much, but it really shines when solving complex coupled problems.

We are solving two problems, sequentially; they are not coupled. We could have done this completely separated, but will here use the same data structures because we can.

### Data structure

We have two equations to solve. Therefore, we need two keywords:
* `flow_keyword = 'flow'`
* `transport_keyword = 'transport'`

The two problems are not coupled, so we will consider them one by one.

The flow problem has one operator and one source term. Recall,
$$-\nabla \cdot (K\nabla p) = f$$
Since we our domain is fractured, we need variables both on the grids and in the mortar cells connecting the grids. Thus, define
* `flow_variable = 'pressure'`
* `flow_variable_edge = 'darcy_flux'`

On the grids, we need a keyword for the diffusion operator and source term,
* `flow_operator_key = 'diffusion'`
* `flow_source_operator_key = 'flow_source'`

The coupling term for the mortar flux between grids is,
$$-\kappa (p_{fracture} - \texttt{tr } p_{matrix}).$$
This term, which is implemented by `RobinCondition`, needs a keyword,
* `coupling_flow_operator_key = 'darcy_flux_key'`

---
The transport problem has three operators and one source term. Recall,
$$ \rho C \left( \frac{\partial T}{\partial t} + u\cdot \nabla T \right) - \nabla\cdot(K_T\nabla T) = g$$
On a fractured domain, we need variables both on grids and for coupling terms. The advection-diffusion problem has mortar flux variables for the diffusive term and the advective term.

We define the grid variable and edge variables,
* `transport_variable = 'tracer'`
* `transport_variable_edge_advection = 'edge_advection'`
* `transport_variable_edge_diffusion = 'edge_diffusion'`


On the grids we define the operator keys,
* `advection_operator_key = 'advection'`
* `mass_operator_key = 'mass'`
* `diffusion_operator_key = 'diffusion_tracer'`
* `transport_source_operator_key = 'transport_source'`
and on the interfaces, we define the operator keys,
* `coupling_transport_advection_operator_key = 'edge_advection_operator_key'`
* `coupling_transport_diffusion_operator_key = 'edge_diffusion_operator_key'`



In [ ]:
# Flow problem:
flow_keyword = 'flow'

flow_variable = 'pressure'
flow_variable_edge = 'darcy_flux'

flow_operator_key = 'diffusion'
flow_source_operator_key = 'flow_source'
coupling_flow_operator_key = 'darcy_flux_key'

# Transport problem
transport_keyword = 'transport'

transport_variable = 'tracer'
transport_variable_edge_advection = 'edge_advection'
transport_variable_edge_diffusion = 'edge_diffusion'

advection_operator_key = 'advection'
mass_operator_key = 'mass'
diffusion_operator_key = 'diffusion_tracer'
transport_source_operator_key = 'transport_source'

coupling_transport_advection_operator_key = 'edge_advection_operator_key'
coupling_transport_diffusion_operator_key = 'edge_diffusion_operator_key'

### Discretization objects
Next, we define discretization objects for the two problems. 

The flow problem uses a `Tpfa` discretization for the diffusion operator and a `ScalarSource` for the source term.

The transport problem is solved using an implicit time-stepping method: Backward Euler. This requires the use of implicit formulations of the discretization methods - essentially multiplying their contributions by the time-step.

* Mass term: $\frac{\partial T}{\partial t}$, use: `ImplicitMassMatrix`
* Advection term: $\mathbf{v} \cdot \nabla T$, use: `ImplicitUpwind`
* Diffusion term: $- \nabla \cdot (K_T \nabla T)$, use: `ImplicitTpfa`
* Source term: $g$, use `ImplicitScalarSource`

Below, `ImplicitTpfa` and `ImplicitScalarSource` are defined. The others are implemented in `utils > derived_discretization > implicit_euler`.

In [ ]:
# Override methods for implicit time-stepping:
class ImplicitTpfa(pp.Tpfa):
    """
    Multiply all contributions by the time step.
    """

    def assemble_matrix_rhs(self, g, data):
        """ Overwrite TPFA method to be consistent with the Biot dt convention.
        """
        a, b = super().assemble_matrix_rhs(g, data)
        dt = data[pp.PARAMETERS][self.keyword]["time_step"]
        a = a * dt
        b = b * dt
        return a, b

    def assemble_int_bound_flux(
        self, g, data, data_edge, grid_swap, cc, matrix, rhs, self_ind
    ):
        """
        Overwrite the TPFA method to be consistent with the Biot dt convention
        """
        dt = data[pp.PARAMETERS][self.keyword]["time_step"]

        div = g.cell_faces.T

        bound_flux = data[pp.DISCRETIZATION_MATRICES][self.keyword]["bound_flux"]
        # Projection operators to grid
        mg = data_edge["mortar_grid"]

        if grid_swap:
            proj = mg.mortar_to_slave_int()
        else:
            proj = mg.mortar_to_master_int()

        if g.dim > 0 and bound_flux.shape[0] != g.num_faces:
            # If bound flux is gven as sub-faces we have to map it from sub-faces
            # to faces
            hf2f = pp.fvutils.map_hf_2_f(nd=1, g=g)
            bound_flux = hf2f * bound_flux
        if g.dim > 0 and bound_flux.shape[1] != proj.shape[0]:
            raise ValueError(
                """Inconsistent shapes. Did you define a
            sub-face boundary condition but only a face-wise mortar?"""
            )

        cc[self_ind, 2] += dt * div * bound_flux * proj
    
    def assemble_int_bound_source(
        self, g, data, data_edge, grid_swap, cc, matrix, rhs, self_ind
    ):
        """ Abstract method. Assemble the contribution from an internal
        boundary, manifested as a source term.
        The intended use is when the internal boundary is coupled to another
        node in a mixed-dimensional method. Specific usage depends on the
        interface condition between the nodes; this method will typically be
        used to impose flux continuity on a lower-dimensional domain.
        Implementations of this method will use an interplay between the grid on
        the node and the mortar grid on the relevant edge.
        Parameters:
            g (Grid): Grid which the condition should be imposed on.
            data (dictionary): Data dictionary for the node in the
                mixed-dimensional grid.
            data_edge (dictionary): Data dictionary for the edge in the
                mixed-dimensional grid.
            grid_swap (boolean): If True, the grid g is identified with the @
                slave side of the mortar grid in data_adge.
            cc (block matrix, 3x3): Block matrix for the coupling condition.
                The first and second rows and columns are identified with the
                master and slave side; the third belongs to the edge variable.
                The discretization of the relevant term is done in-place in cc.
            matrix (block matrix 3x3): Discretization matrix for the edge and
                the two adjacent nodes.
            rhs (block_array 3x1): Right hand side contribution for the edge and
                the two adjacent nodes.
            self_ind (int): Index in cc and matrix associated with this node.
                Should be either 1 or 2.
        """
        mg = data_edge["mortar_grid"]

        if grid_swap:
            proj = mg.mortar_to_master_int()
        else:
            proj = mg.mortar_to_slave_int()
        dt = data[pp.PARAMETERS][self.keyword]["time_step"]
        cc[self_ind, 2] -= proj * dt


class ImplicitScalarSource(pp.ScalarSource):
    """
    Multiply right-hand side source vector by the time step.
    """
    
    def assemble_rhs(self, g, data):
        """ Return the rhs for a discretization of the integrated source term. Also
        discretize the necessary operators if the data dictionary does not contain a
        source term.

        Parameters:
            g (Grid): Computational grid, with geometry fields computed.
            data (dictionary): With data stored.

        Returns:
            np.array (self.ndof): Right hand side vector representing the
                source.

        """
        dt = data[pp.PARAMETERS][self.keyword]["time_step"]
        
        matrix_dictionary = data[pp.DISCRETIZATION_MATRICES][self.keyword]
        parameter_dictionary = data[pp.PARAMETERS][self.keyword]

        sources = parameter_dictionary["source"]
        assert sources.size == self.ndof(
            g
        ), "There should be one source value for each cell"
        return matrix_dictionary["bound_source"] * sources * dt

In [ ]:
# Flow problem
flow_discretization = pp.Tpfa(flow_keyword)
edge_discretization_flow = pp.RobinCoupling(flow_keyword, 
                                            flow_discretization, 
                                            flow_discretization)

# Transport problem
mass_discretization = IE_discretizations.ImplicitMassMatrix(keyword=transport_keyword,
                                                            variable=transport_variable)
advection_discretization = IE_discretizations.ImplicitUpwind(transport_keyword)
diffusion_discretization = ImplicitTpfa(transport_keyword)

edge_discretization_diffusion = pp.RobinCoupling(transport_keyword,
                                                 diffusion_discretization,
                                                 diffusion_discretization)
edge_discretization_advection = IE_discretizations.ImplicitUpwindCoupling(transport_keyword)

for g, d in gb:
    